In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
search_query="titan+men+watches"

In [3]:
base_url="https://www.amazon.in/s?k="

In [4]:
url=base_url+search_query

In [ ]:
url

In [6]:
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

In [7]:
search_response=requests.get(url,headers=header)

In [ ]:
search_response.status_code

In [ ]:
search_response.text

#### A function to get the content of the page of required query

In [10]:
cookie={} # insert request cookies within{}
def getAmazonSearch(search_query):
    url="https://www.amazon.in/s?k="+search_query
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

#### A function to get the contents of individual product pages using 'data-asin' number (unique identification number)

In [11]:
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

#### A function to pass on the link of 'see all reviews' and extract the content

In [12]:
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

### First page product reviews extraction

In [ ]:
product_names=[]
response=getAmazonSearch('titan+men+watches')
soup=BeautifulSoup(response.content)
for i in soup.findAll("span",{'class':'a-size-base-plus a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list

findall() is used to find all the tags with the mentioned attributes and values in a page

In [ ]:
product_names

In [15]:
len(product_names)

61

##### The method of extracting data-asin numbers are similar to that of product names. Only the tag details have to be changed in findall()

In [16]:
data_asin=[]
response=getAmazonSearch('titan+men+watches')
soup=BeautifulSoup(response.content)
for i in soup.findAll("div",{'class':"sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32"}):
    data_asin.append(i['data-asin'])

https://www.amazon.in/s?k=titan+men+watches


In [ ]:
data_asin

In [18]:
len(data_asin)

48

##### By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [ ]:
link=[]
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

In [20]:
len(link)

46

##### Now we have the 'see all review' links. Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [ ]:
reviews=[]
for j in range(len(link)):
    for k in range(100):
        response=Searchreviews(link[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text)

In [23]:
len(reviews)

3765

In [ ]:
reviews[:40]

In [26]:
import pandas as pd

### Second page product reviews extraction

##### Repeating the same procedure for page 2 of the products. '&page=2' has to be passed along with the search query to get data-asin numbers on 2nd page

##### Deleting the elements in data-asin[] and link[] so that the same data does not repeat

In [27]:
del data_asin[:]

In [28]:
response=getAmazonSearch('titan+men+watches&page=2')
soup=BeautifulSoup(response.content)
for i in soup.findAll("div",{'class':"sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32"}):
    data_asin.append(i['data-asin'])

https://www.amazon.in/s?k=titan+men+watches&page=2


In [29]:
len(data_asin)

53

In [30]:
del link[:]

In [31]:
len(link)

0

In [ ]:
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

In [33]:
len(link)

51

In [ ]:
for j in range(len(link)):
    for k in range(100):
        response=Searchreviews(link[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text)

In [35]:
len(reviews)

6194

In [36]:
rev={'reviews':reviews} #converting the reviews list into a dictionary

In [37]:
review_data=pd.DataFrame.from_dict(rev) #converting this dictionary into a dataframe

In [39]:
review_data.sample(5)

,reviews
5334,Good product\n
2502,Good quality\n
5736,perfect watch ! excellent built quality by tit...
4822,very awesome and best\n
2537,Love this watch\n


In [40]:
review_data.shape

(6194, 1)

In [41]:
review_data.to_csv('Scraping reviews.csv',index=False) #converting the dataframe to a csv file so as to use it later for further analysis